# Progetto di Madon Federica 825628

### Importazione dei pacchetti e dei dataset. Informazioni dataset.

In [1]:
import pandas as pd

In [2]:
#Le opzioni in read_csv sono per unzippare i file
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', encoding = 'latin-1', compression = 'gzip', header = 0, sep = ';', quotechar = '"', low_memory = False)
books_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', encoding = 'latin-1', compression = 'gzip', header = 0, sep = ';', quotechar = '"', low_memory = False)
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', encoding = 'latin-1', compression = 'gzip', header = 0, sep = ';', quotechar = '"', low_memory = False)
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', encoding = 'latin-1', compression = 'gzip', header = 0, sep = ',', quotechar = '"', low_memory = False)
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', encoding = 'latin-1', compression = 'gzip', header = 0, sep = ',', quotechar = '"', low_memory = False)

In [3]:
#Controllo dimensioni dataset
print(f"Dimensioni books: {books.shape}")
print(f"Dimensioni books_ratings: {books_ratings.shape}")
print(f"Dimensioni users: {users.shape}")
print(f"Dimensioni goodbooks: {goodbooks.shape}")
print(f"Dimensioni goodbooks_ratings: {goodbooks_ratings.shape}")

Dimensioni books: (271359, 8)
Dimensioni books_ratings: (1149780, 3)
Dimensioni users: (278858, 3)
Dimensioni goodbooks: (99, 23)
Dimensioni goodbooks_ratings: (99, 3)


In seguito analizzo le informazioni dei 5 dataset.

In [4]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271359 entries, 0 to 271358
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271359 non-null  object
 1   Book-Title           271359 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271359 non-null  object
 4   Publisher            271357 non-null  object
 5   Image-URL-S          271359 non-null  object
 6   Image-URL-M          271359 non-null  object
 7   Image-URL-L          271356 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [5]:
books_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [7]:
goodbooks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    99 non-null     int64  
 1   goodreads_book_id          99 non-null     int64  
 2   best_book_id               99 non-null     int64  
 3   work_id                    99 non-null     int64  
 4   books_count                99 non-null     int64  
 5   isbn                       99 non-null     object 
 6   isbn13                     99 non-null     float64
 7   authors                    99 non-null     object 
 8   original_publication_year  99 non-null     float64
 9   original_title             98 non-null     object 
 10  title                      99 non-null     object 
 11  language_code              98 non-null     object 
 12  average_rating             99 non-null     float64
 13  ratings_count              99 non-null     int64  
 

In [8]:
goodbooks_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  99 non-null     int64
 1   book_id  99 non-null     int64
 2   rating   99 non-null     int64
dtypes: int64(3)
memory usage: 2.4 KB


### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [9]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


Il campo Location contiene dei valori con dei problemi di consistenza, nel senso che non tutte le righe hanno questo attributo scritto correttamente come città, regione, nazione. Il numero di virgole mi dice in totale quante stringhe ci sono in una cella. Ci sono più di 1500 campi con più di due virgole e due con meno.

In [10]:
#Più di tre stringhe nel campo location
len(users['Location'][(users['Location'].str.count(',')) > 2].tolist())

1508

In [11]:
#Controllo se tra questi 1508 ci sono duplicati
len(users['Location'][(users['Location'].str.count(',')) > 2].unique().tolist())

1398

In [12]:
#Meno di tre stringhe nel campo location
len(users['Location'][(users['Location'].str.count(',')) < 2].tolist())

2

In [13]:
#Visualizzo le righe con meno virgole
users.iloc[users['Location'][(users['Location'].str.count(',')) < 2].index]

,User-ID,Location,Age
29419,29420,"st. louis, missouri",68.0
134376,134377,lawrenceville,30.0


In [14]:
#Correggo le due righe in esame
users.loc[29419, 'Location'] = 'st. louis, st. louis, missouri'
users.loc[134376, 'Location'] = 'lawrenceville, georgia, usa'
#Ricontrollo
users.iloc[[29419, 134376]]

,User-ID,Location,Age
29419,29420,"st. louis, st. louis, missouri",68.0
134376,134377,"lawrenceville, georgia, usa",30.0


In [15]:
#Visualizzo se ho corretto queste due righe
users.iloc[users['Location'][(users['Location'].str.count(',')) < 2].index]

,User-ID,Location,Age


Per quanto riguarda i campi con più virgole ci sono tre opzioni:
1.   Eliminare le righe, considerando la mole del dataset;
2.   Ignorare il problema;
3.   Correggere i 1500 e passa valori.

Ho deciso di eliminare le righe, perché correggere tutti valori non mi sembra il caso (in ambito lavorativo avrei dovuto pensarci di più). Presento, inoltre, anche l'opzione due, anche se è un errore, poiché non so se posso eliminare le righe.

In [16]:
#Decido di eliminare quelle con numero di virgole maggiori considerando la mole del dataset
users_elimin = users.drop(users.index[users['Location'].str.count(',') > 2])
#Ricontrollo il problema delle virgole nel nuovo dataset
users_elimin.iloc[users_elimin['Location'][(users_elimin['Location'].str.count(',')) > 2]]

,User-ID,Location,Age


In [17]:
#Divido la stringa e salvo i vari pezzi in un dataframe con tre colonne diverse
divise = users_elimin.Location.str.split(',', expand = True)
users_elimin['City'] = divise[0]
users_elimin['Region'] = divise[1]
users_elimin['Country'] = divise[2]
#Resetto l'indice perché ho eliminato delle righe
users_elimin.reset_index(inplace = True, drop = True)
users_elimin.head(2)

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa


In [18]:
#Faccio la stessa cosa con quello intero
divise = users.Location.str.split(',', expand = True)
users['City'] = divise[0]
users['Region'] = divise[1]
users['Country'] = divise[2]
users.head(2)

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa


### 2. For each book in the *Books* dataset, compute its average rating.

In [19]:
#Controllo se le due tabelle books e books_ratings hanno valori unici per quanto riguarda l'ISBN 
#rispetto alla dimensione totale dei dataset
print(f"Dimensione tabella books: {len(books)}")
print(f"Totale valori unici ISBN tabella books: {len(books['ISBN'].unique())}")
print(f"Dimensione tabella books_ratings: {len(books_ratings)}")
print(f"Totale valori unici ISBN tabella books_ratings: {len(books_ratings['ISBN'].unique())}")

Dimensione tabella books: 271359
Totale valori unici ISBN tabella books: 271359
Dimensione tabella books_ratings: 1149780
Totale valori unici ISBN tabella books_ratings: 340556


Perciò nella tabella books_ratings ci sono più votazioni per un singolo libro.

In [20]:
#Controllo se i valori di ISBN sono gli stessi per le due tabelle
#Uso unique() solo per una tabella, perché l'altra ha tutti valori unici per ISBN
print(f"Differenza numero valori ISBN di books e books_ratings: {len(set(books['ISBN'].unique()) - set(books_ratings['ISBN']))}")
print(f"Differenza numero valori ISBN di books_ratings e books: {len(set(books_ratings['ISBN'].unique()) - set(books['ISBN']))}")

Differenza numero valori ISBN di books e books_ratings: 1209
Differenza numero valori ISBN di books_ratings e books: 70406


In [21]:
#Calcolo nella tabella books_ratings i valori medi del rating per ogni ISBN
books_avg_ratings = books_ratings.groupby('ISBN').mean('Book-Rating')[['Book-Rating']]
#Cambio nome colonna
books_avg_ratings.rename(columns = {'Book-Rating' : 'Avg-Rating'}, inplace = True)
#Resetto l'indice
books_avg_ratings.reset_index(inplace = True)
books_avg_ratings.head(2)

,ISBN,Avg-Rating
0,0330299891,3.0
1,0375404120,1.5


In [22]:
#Faccio il merge tra il dataset books e quello nuovo, che contiene la media dei voti perché non hanno 
#gli stessi valori per ISBN.
merge_books_avg_ratings = books.merge(books_avg_ratings, on = 'ISBN', how = 'left')
merge_books_avg_ratings.head(2)
#Ho usato left perché dice per ogni

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Avg-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.928571


In [23]:
#In questo caso non avrò valori nulli per quanto riguarda il punteggio medio
merge_books_avg_noleft = books.merge(books_avg_ratings, on = 'ISBN')
merge_books_avg_noleft.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Avg-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.928571


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [24]:
#Controllo se le due tabelle goodbooks e goodbooks_ratings hanno valori unici per quanto riguarda 
#book_id rispetto alla dimensione totale dei dataset
print(f"Dimensione tabella goodbooks: {len(goodbooks)}")
print(f"Totale valori unici book_id tabella goodbooks: {len(goodbooks['book_id'].unique())}")
print(f"Dimensione tabella goodbooks_ratings: {len(goodbooks_ratings)}")
print(f"Totale valori unici book_id tabella goodbooks_ratings: {len(goodbooks_ratings['book_id'].unique())}")

Dimensione tabella goodbooks: 99
Totale valori unici book_id tabella goodbooks: 99
Dimensione tabella goodbooks_ratings: 99
Totale valori unici book_id tabella goodbooks_ratings: 96


In [25]:
#Controllo se i valori di book_id sono gli stessi per le due tabelle
#Uso unique() solo per una tabella, perché l'altra ha tutti valori unici per book_id
print(f"Differenza numero valori book_id di goodbooks e goodbooks_ratings: {len(set(goodbooks['book_id'].unique()) - set(goodbooks_ratings['book_id']))}")
print(f"Differenza numero valori book_id di goodbooks_ratings e goodbooks: {len(set(goodbooks_ratings['book_id'].unique()) - set(goodbooks['book_id']))}")

Differenza numero valori book_id di goodbooks e goodbooks_ratings: 72
Differenza numero valori book_id di goodbooks_ratings e goodbooks: 69


Perciò i due dataset hanno valori praticamente diversi, se non per circa 20, di book_id.

In [26]:
#Calcolo nella tabella goodbooks_ratings il valore medio di rating per ogni book_id
avg_goodbooks = goodbooks_ratings.groupby('book_id')[['rating']].mean()
#Cambio il nome di una colonna
avg_goodbooks.rename(columns = {'rating' : 'avg_rating'}, inplace = True)
#Resetto l'indice
avg_goodbooks.reset_index(inplace = True)
avg_goodbooks.head(2)

,book_id,avg_rating
0,2,5.0
1,5,4.0


In [27]:
#Faccio il merge tra il dataset goodbooks e quello nuovo, che contiene la media dei voti perché 
#non hanno gli stessi valori per book_id
merge_goodbooks_avg_ratings = goodbooks.merge(avg_goodbooks, on = 'book_id', how = 'left')
merge_goodbooks_avg_ratings.head(2)
#Uso left per tenere i valori della tabella goodbooks, ovviamente avrò valori nulli per molti rating

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,avg_rating
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,NaN
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,5.0


In [28]:
#Se non voglio avere valori nulli per il rating
merge_goodbooks_avg_noleft = goodbooks.merge(avg_goodbooks, on = 'book_id')
merge_goodbooks_avg_noleft.head(2)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,avg_rating
0,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,5.0
1,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,4.0


In [29]:
#Poiché i valori di book_id sono diversi per le due tabella goodbooks e goodbooks_ratings, mostro 
#anche i valori di average_rating presenti di default nella tabella goodbooks
goodbooks[['book_id', 'isbn', 'authors', 'original_publication_year', 'original_title', 'average_rating']].head(2)

,book_id,isbn,authors,original_publication_year,original_title,average_rating
0,1,439023483,Suzanne Collins,2008.0,The Hunger Games,4.34
1,2,439554934,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,4.44


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [30]:
#Raggruppo i libri per Book-Title, Book-Author e Publisher e conteggio quanti ISBN hanno per ognuno 
#(un libro che viene ripubblicato cambia ISBN)
merged_books = books.groupby(['Book-Title', 'Book-Author', 'Publisher']).count()[['ISBN']]
#Sistemo l'indice, poiché è più comodo non lavorare con i multi-indici
merged_books.reset_index(inplace = True)
merged_books.head(2)

,Book-Title,Book-Author,Publisher,ISBN
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,Hyperion Books for Children,1
1,Always Have Popsicles,Rebecca Harvin,Rebecca L. Harvin,1


In [31]:
#Elimino i libri che non sono stati ripubblicati (quindi ISBN <= 1)
merged_books.drop(merged_books.index[merged_books['ISBN'] <= 1], inplace = True)
#Elimino colonna ISBN perché non mi serve
merged_books.drop(columns = ['ISBN'], inplace = True)
#Sistemo l'indice avendo eliminato delle righe
merged_books.reset_index(drop = True, inplace = True)
merged_books.head(2)

,Book-Title,Book-Author,Publisher
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic


### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [32]:
#Considero la tabella merge_books_avg_ratings che contiene i libri della tabella books e per quasi 
#tutti il valore di avg_ratings
#Faccio un merge tra la tabella considerata e merged_books, ottenendo i libri in comune a tutte e due
merged_books_avg_ratings = merge_books_avg_ratings.merge(merged_books, on=['Book-Title', 'Book-Author', 'Publisher'])
#Ordino i valori del nuovo dataframe in ordine alfabetico rispetto ai tre attributi
merged_books_avg_ratings.sort_values(by = ['Book-Title', 'Book-Author', 'Publisher'], inplace = True)
#Resetto l'indice
merged_books_avg_ratings.reset_index(drop = True, inplace = True)
merged_books_avg_ratings[['ISBN', 'Book-Title', 'Book-Author', 'Publisher', 'Avg-Rating']].head(2)

,ISBN,Book-Title,Book-Author,Publisher,Avg-Rating
0,1565920465,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,0.0
1,1565920317,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.0


Anche qui potevo considerare anche il dataset merge_book_avg_noleft, che non ha valori nulli di Avg-Rating, ma sarebbe la stessa procedura.

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [33]:
#Considero la tabella merge_books_avg_ratings che contiene i libri della tabella books e per quasi 
#tutti il valore di avg_ratings
#Raggruppo e calcolo il massimo del punteggio medio
merged_books_splitted_ratings = merge_books_avg_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher']).max('Avg-Rating')
#Cambio il nome di una colonna
merged_books_splitted_ratings.rename(columns = {'Avg-Rating' : 'Max-Rating'}, inplace = True)
#Aggiungo una colonna con il minimo
merged_books_splitted_ratings['Min-Rating'] = merge_books_avg_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher']).min('Avg-Rating')
#Aggiungo una colonna con il conteggio dei libri
merged_books_splitted_ratings['Count'] = merge_books_avg_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher']).count()['ISBN']
#Resetto l'indice
merged_books_splitted_ratings.reset_index(inplace = True)
#Elimino le righe con i libri che erano presenti solo una volta
merged_books_splitted_ratings.drop(merged_books_splitted_ratings.index[merged_books_splitted_ratings['Count'] <= 1], inplace = True)
#Elimino colonna inutile
merged_books_splitted_ratings.drop(columns = ['Count'], inplace = True)
#Resetto di nuovo l'indice
merged_books_splitted_ratings.reset_index(drop = True, inplace = True)
merged_books_splitted_ratings.head(2)

,Book-Title,Book-Author,Publisher,Max-Rating,Min-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.0,0.0
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,3.4,0.0


### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [34]:
#Poiché la colonna authors contiene più autori
#Aggiungo una colonna con il numero di autori 
goodbooks['num_author'] = goodbooks['authors'].str.count(',') + 1
#Divido la colonna reviews_count_per_author per il numero di autori
goodbooks['reviews_count_per_author'] = (goodbooks['work_text_reviews_count']) / (goodbooks['num_author'])
goodbooks.head(2)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,num_author,reviews_count_per_author
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,1,155254.0
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2,37933.5


In [35]:
#Salvo le stringhe dei vari autori in un dataframe nuovo
authors_divided = goodbooks.authors.str.split(',', expand = True)
#Faccio una copia del dataset goodbooks per non modificarlo direttamente
goodbooks_authors_divided = goodbooks.copy()
#Inserisco i valori degli autori nella tabella
goodbooks_authors_divided['authors'] = authors_divided[0]
goodbooks_authors_divided.rename(columns = {'authors': 'authors_1'}, inplace = True)
goodbooks_authors_divided['authors_2'] = authors_divided[1]
goodbooks_authors_divided['authors_3'] = authors_divided[2]
goodbooks_authors_divided['authors_4'] = authors_divided[3]
goodbooks_authors_divided['authors_5'] = authors_divided[4]

In [36]:
#Salvo in 5 dataframe gli autori con la somma delle reviews_count_per_author
author_1 = goodbooks_authors_divided.groupby('authors_1')[['reviews_count_per_author']].sum('reviews_count_per_author')
author_2 = goodbooks_authors_divided.groupby('authors_2')[['reviews_count_per_author']].sum('reviews_count_per_author')
author_3 = goodbooks_authors_divided.groupby('authors_3')[['reviews_count_per_author']].sum('reviews_count_per_author')
author_4 = goodbooks_authors_divided.groupby('authors_4')[['reviews_count_per_author']].sum('reviews_count_per_author')
author_5 = goodbooks_authors_divided.groupby('authors_5')[['reviews_count_per_author']].sum('reviews_count_per_author')

In [37]:
#Cambio il nome della colonna
author_1.rename(columns = {'reviews_count_per_author' : 'Reviews_with_text'}, inplace = True)
author_2.rename(columns = {'reviews_count_per_author' : 'Reviews_with_text'}, inplace = True)
author_3.rename(columns = {'reviews_count_per_author' : 'Reviews_with_text'}, inplace = True)
author_4.rename(columns = {'reviews_count_per_author' : 'Reviews_with_text'}, inplace = True)
author_5.rename(columns = {'reviews_count_per_author' : 'Reviews_with_text'}, inplace = True)

In [38]:
#Resetto l'indice
author_1.reset_index(inplace = True)
author_2.reset_index(inplace = True)
author_3.reset_index(inplace = True)
author_4.reset_index(inplace = True)
author_5.reset_index(inplace = True)

In [39]:
#Cambio il nome della colonna
author_1.rename(columns = {'authors_1' : 'Authors'}, inplace = True)
author_2.rename(columns = {'authors_2' : 'Authors'}, inplace = True)
author_3.rename(columns = {'authors_3' : 'Authors'}, inplace = True)
author_4.rename(columns = {'authors_4' : 'Authors'}, inplace = True)
author_5.rename(columns = {'authors_5' : 'Authors'}, inplace = True)
#Concateno i dataset degli autori
authors = pd.concat([author_1, author_2, author_3, author_4, author_5])
authors.head(2)

,Authors,Reviews_with_text
0,Aldous Huxley,20095.0
1,Alice Sebold,36642.0


In [40]:
#Dato che i vari dataset degli autori presentavano anche gli stessi valori devo raggruppare di nuovo
authors_reviews = authors.groupby('Authors')[['Reviews_with_text']].sum('Reviews_with_text')
authors_reviews.head(2)

,Reviews_with_text
Authors,
Alan R. Clarke,27890.500000
B.M. Mooyaart-Doubleday,6941.666667


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [41]:
#Considero il dataset goodbooks_authors_divided
#Controllo gli anni di pubblicazione
goodbooks_authors_divided['original_publication_year'].unique().tolist()

[2008.0,
 1997.0,
 2005.0,
 1960.0,
 1925.0,
 2012.0,
 1937.0,
 1951.0,
 2000.0,
 1813.0,
 2003.0,
 2011.0,
 1949.0,
 1945.0,
 1947.0,
 2009.0,
 1999.0,
 1954.0,
 2010.0,
 2002.0,
 1998.0,
 2007.0,
 1595.0,
 1988.0,
 1993.0,
 1950.0,
 1996.0,
 2006.0,
 1868.0,
 1847.0,
 2001.0,
 1953.0,
 1974.0,
 1979.0,
 1932.0,
 1884.0,
 1952.0,
 2015.0,
 1995.0,
 2004.0,
 1969.0,
 1936.0,
 1985.0,
 1818.0,
 1977.0,
 1811.0,
 -720.0,
 1946.0,
 1859.0,
 1990.0,
 1964.0,
 1989.0,
 1958.0,
 1973.0,
 1967.0,
 1911.0,
 1891.0,
 1897.0]

In [42]:
#Faccio le stesse operazioni del punto 7 considerando però anche gli anni
author1_year = goodbooks_authors_divided.groupby(['original_publication_year', 'authors_1'])[['reviews_count_per_author']].sum()
author2_year = goodbooks_authors_divided.groupby(['original_publication_year', 'authors_2'])[['reviews_count_per_author']].sum()
author3_year = goodbooks_authors_divided.groupby(['original_publication_year', 'authors_3'])[['reviews_count_per_author']].sum()
author4_year = goodbooks_authors_divided.groupby(['original_publication_year', 'authors_4'])[['reviews_count_per_author']].sum()
author5_year = goodbooks_authors_divided.groupby(['original_publication_year', 'authors_5'])[['reviews_count_per_author']].sum()

In [43]:
#Resetto l'indice
author1_year.reset_index(inplace = True)
author2_year.reset_index(inplace = True)
author3_year.reset_index(inplace = True)
author4_year.reset_index(inplace = True)
author5_year.reset_index(inplace = True)

In [44]:
#Cambio i nomi delle colonne
author1_year.rename(columns = {'original_publication_year' : 'Publication_Year', 'authors_1' : 'Authors'}, inplace = True)
author2_year.rename(columns = {'original_publication_year' : 'Publication_Year', 'authors_2' : 'Authors'}, inplace = True)
author3_year.rename(columns = {'original_publication_year' : 'Publication_Year', 'authors_3' : 'Authors'}, inplace = True)
author4_year.rename(columns = {'original_publication_year' : 'Publication_Year', 'authors_4' : 'Authors'}, inplace = True)
author5_year.rename(columns = {'original_publication_year' : 'Publication_Year', 'authors_5' : 'Authors'}, inplace = True)

In [45]:
#Concateno i dataset
authors_year = pd.concat([author1_year, author2_year, author3_year, author4_year, author5_year])
#Calcolo il max
authors_year_max = authors_year.groupby(['Publication_Year']).max()
#Cambio nome colonna
authors_year_max.rename(columns = {'reviews_count_per_author' : 'Max_reviews'}, inplace = True)
#Resetto l'indice
authors_year_max.reset_index(inplace = True)
authors_year_max.head(2)

,Publication_Year,Authors,Max_reviews
0,-720.0,Homer,1620.2
1,1595.0,William Shakespeare,7389.0


### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [46]:
#Considero i dataset: merge_books_avg_ratings e merge_goodbooks_avg_ratings, ovviamente i valori nulli
#danno problemi nel fare la differenza
both_books_diff_avg = pd.merge(merge_books_avg_ratings, merge_goodbooks_avg_ratings, left_on = 'ISBN', right_on = 'isbn')
both_books_diff_avg['Diff_rating_goodrating'] = both_books_diff_avg['Avg-Rating'] - both_books_diff_avg['avg_rating']
both_books_diff_avg.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Avg-Rating,book_id,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,avg_rating,Diff_rating_goodrating
0,1400032717,The Curious Incident of the Dog in the Night-T...,Mark Haddon,2004,Vintage,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,4.813187,60,...,35796,21049,58117,218218,355925,262481,https://images.gr-assets.com/books/1479863624m...,https://images.gr-assets.com/books/1479863624s...,NaN,NaN
1,014028009X,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,3.751852,75,...,8157,28973,59309,198555,254836,212825,https://images.gr-assets.com/books/1292060045m...,https://images.gr-assets.com/books/1292060045s...,NaN,NaN


In [47]:
#Considero i dataset: merge_book_avg_noleft e merge_goodbooks_avg_noleft, in questo caso non ci sono
#valori nulli
both_books_diff_nonull = pd.merge(merge_books_avg_noleft, merge_goodbooks_avg_noleft, left_on = 'ISBN', right_on = 'isbn')
both_books_diff_nonull['Diff_rating_goodrating'] = both_books_diff_nonull['Avg-Rating'] - both_books_diff_nonull['avg_rating']
both_books_diff_nonull.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Avg-Rating,book_id,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,avg_rating,Diff_rating_goodrating
0,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,2004,Scholastic Paperbacks,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,3.533333,18,...,36099,6716,20413,166129,509447,1266670,https://images.gr-assets.com/books/1499277281m...,https://images.gr-assets.com/books/1499277281s...,5.0,-1.466667


### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [48]:
#Divido il dataset in vari dataframe in base all'età
users_na = users[users['Age'].isnull()] #non hanno l'età
users0_14 = users[(users['Age'] >= 0) & (users['Age'] <= 14)] #età 0-14
users15_24 = users[(users['Age'] >= 15) & (users['Age'] <= 24)] #età 15-24
users25_34 = users[(users['Age'] >= 25) & (users['Age'] <= 34)] #età 25-34
users35_44 = users[(users['Age'] >= 35) & (users['Age'] <= 44)] #età 35-44
users45_54 = users[(users['Age'] >= 45) & (users['Age'] <= 54)] #età 45-54
users55_64 = users[(users['Age'] >= 55) & (users['Age'] <= 64)] #età 55-64
users65_74 = users[(users['Age'] >= 65) & (users['Age'] <= 74)] #età 65-74
users75_84 = users[(users['Age'] >= 75) & (users['Age'] <= 84)] #età 75-84
users85_94 = users[(users['Age'] >= 85) & (users['Age'] <= 94)] #età 85-94
users95_104 = users[(users['Age'] >= 95) & (users['Age'] <= 104)] #età 95-104

In [49]:
print(f"Gli utenti senza età sono: {len(users_na)}")
print(f"Gli utenti con età 0-14 sono: {len(users0_14)}")
print(f"Gli utenti con età 15-24 sono: {len(users15_24)}")
print(f"Gli utenti con età 25-34 sono: {len(users25_34)}")
print(f"Gli utenti con età 35-44 sono: {len(users35_44)}")
print(f"Gli utenti con età 45-54 sono: {len(users45_54)}")
print(f"Gli utenti con età 55-64 sono: {len(users55_64)}")
print(f"Gli utenti con età 65-74 sono: {len(users65_74)}")
print(f"Gli utenti con età 75-84 sono: {len(users75_84)}")
print(f"Gli utenti con età 85-94 sono: {len(users85_94)}")
print(f"Gli utenti con età 95-104 sono: {len(users95_104)}")

Gli utenti senza età sono: 110762
Gli utenti con età 0-14 sono: 4313
Gli utenti con età 15-24 sono: 40001
Gli utenti con età 25-34 sono: 50767
Gli utenti con età 35-44 sono: 32690
Gli utenti con età 45-54 sono: 23152
Gli utenti con età 55-64 sono: 12493
Gli utenti con età 65-74 sono: 3596
Gli utenti con età 75-84 sono: 615
Gli utenti con età 85-94 sono: 83
Gli utenti con età 95-104 sono: 278


### 11. Find the books that appear only in the goodbooks datasets.

In [50]:
#Faccio un left outer join tra la tabella goodbooks e books, left perché voglio mantenere tutti i 
#libri di goodbooks
goodbooks_only = goodbooks.merge(books, left_on = 'isbn', right_on = 'ISBN', how = 'left')
#Elimino le righe che sono in comune con l'altro dataset, quindi quelle con ISBN non nullo
goodbooks_only.drop(goodbooks_only.index[goodbooks_only['ISBN'].notnull()], inplace = True)
#Resetto l'indice
goodbooks_only.reset_index(inplace = True, drop = True)
goodbooks_only[['book_id', 'isbn', 'authors', 'original_publication_year', 'original_title']].head(2)

,book_id,isbn,authors,original_publication_year,original_title
0,1,439023483,Suzanne Collins,2008.0,The Hunger Games
1,2,439554934,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone


### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [51]:
#Raggruppo i libri per Book-Author e Book-Title e conto quanti ISBN ci sono
books_authors_title = books.groupby(['Book-Author', 'Book-Title'])[['ISBN']].count()
#Cambio il nome di una colonna
books_authors_title.rename(columns = {'ISBN' : 'Num_time_appear'}, inplace = True)
#Riordino il dataset in ordine decrescente in base ai valori di Num_time_appear
books_authors_title.sort_values(by='Num_time_appear',ascending = False, inplace = True)
books_authors_title.head(10)

,,Num_time_appear
Book-Author,Book-Title,
Louisa May Alcott,Little Women,21
Emily Bronte,Wuthering Heights,20
Mark Twain,Adventures of Huckleberry Finn,20
Jane Austen,Pride and Prejudice,18
Frances Hodgson Burnett,The Secret Garden,16
Bram Stoker,Dracula,16
Charlotte Bronte,Jane Eyre,16
Charles Dickens,Great Expectations,16
Anna Sewell,Black Beauty,14


### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [52]:
#Come fatto in precedenza raggruppo per i vari autori in 5 diversi dataframe e faccio la media della
#colonna average_ratings (già presente nel dataset goodbooks)
auth1 = goodbooks_authors_divided.groupby('authors_1')[['average_rating']].mean()
auth2 = goodbooks_authors_divided.groupby('authors_2')[['average_rating']].mean()
auth3 = goodbooks_authors_divided.groupby('authors_3')[['average_rating']].mean()
auth4 = goodbooks_authors_divided.groupby('authors_4')[['average_rating']].mean()
auth5 = goodbooks_authors_divided.groupby('authors_5')[['average_rating']].mean()

In [53]:
#Resetto l'indice
auth1.reset_index(inplace = True)
auth2.reset_index(inplace = True)
auth3.reset_index(inplace = True)
auth4.reset_index(inplace = True)
auth5.reset_index(inplace = True)

In [54]:
#Cambio il nome di una colonna
auth1.rename(columns = {'authors_1' : 'Authors'}, inplace = True)
auth2.rename(columns = {'authors_2' : 'Authors'}, inplace = True)
auth3.rename(columns = {'authors_3' : 'Authors'}, inplace = True)
auth4.rename(columns = {'authors_4' : 'Authors'}, inplace = True)
auth5.rename(columns = {'authors_5' : 'Authors'}, inplace = True)

In [55]:
#Concateno i dataset in un unico
authors_avg = pd.concat([auth1, auth2, auth3, auth4, auth5])
#Raggruppo di nuovo e ricalcolo la media perché potrebbero esserci gli stessi autori
authors_avg_ratings = authors_avg.groupby('Authors')[['average_rating']].mean()
#Resetto l'indice
authors_avg_ratings.reset_index(inplace = True)
authors_avg_ratings.head(2)

,Authors,average_rating
0,Alan R. Clarke,3.82
1,B.M. Mooyaart-Doubleday,4.10


In [56]:
#L'autore con il massimo valore di average_rating
authors_avg_ratings.iloc[[authors_avg_ratings['average_rating'].idxmax()]]

,Authors,average_rating
32,Rufus Beck,4.53
